In [47]:
import numpy as np
import pandas as pd 
from tqdm import tqdm 
from functools import lru_cache
import os 
import copy

In [48]:
recstudio_data_dir = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio'
raw_data_dir = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data'

In [56]:

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(os.path.join(recstudio_data_dir, 'UK_data/UK_valid_sessions_phase2.csv'))

@lru_cache(maxsize=1)
def read_DE_valid_data():
    return pd.read_csv(os.path.join(recstudio_data_dir, 'DE_data/DE_valid_sessions_phase2.csv'))

@lru_cache(maxsize=1)
def read_JP_valid_data():
    return pd.read_csv(os.path.join(recstudio_data_dir, 'JP_data/JP_valid_sessions_phase2.csv'))

@lru_cache(maxsize=1)
def read_UK_valid_data():
    return pd.read_csv(os.path.join(recstudio_data_dir, 'UK_data/UK_valid_sessions_phase2.csv'))

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(os.path.join(raw_data_dir, 'sessions_test_task1.csv'))

@lru_cache(maxsize=1)
def read_valid_300_prediction():
    return pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_title_results_UK/valid_results/phase2_task1_valid_300_with_score_UK.parquet')

@lru_cache(maxsize=1)
def read_test_300_prediction():
    return pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_xlm_roberta_results/test_results/phase2_task1_test_300_with_score.parquet')

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/processed_products_train.csv')

In [50]:
def filter_hist(sessions_df, prediction_df, num_candidates):
    assert len(sessions_df) == len(prediction_df)
    filtered_prediction_df = copy.deepcopy(prediction_df)
    for row in tqdm(prediction_df.itertuples(), total=len(prediction_df)):
        idx = row.Index

        new_candidates = []
        if 'scores' in row._fields:
            new_scores = []
        
        hist_candidates = eval(sessions_df.iloc[idx]['prev_items'].replace(' ', ','))
        candidates = row.next_item_prediction
        if 'scores' in row._fields:
            scores = row.scores
        for j in range(len(candidates)):
            if candidates[j] not in hist_candidates:
                new_candidates.append(candidates[j])
                if 'scores' in row._fields:
                    new_scores.append(scores[j])

        new_candidates = new_candidates[:num_candidates]
        if 'scores' in row._fields:
            new_scores = new_scores[:num_candidates]
        
        filtered_prediction_df.iloc[idx]['next_item_prediction'] = np.array(new_candidates, dtype=object)
        if 'scores' in row._fields:
            filtered_prediction_df.iloc[idx]['scores'] = np.array(new_scores, dtype=object)
    return filtered_prediction_df

# valid prediction

In [51]:
product_data = read_product_data()

In [52]:
valid_sessions_df = read_valid_data()
valid_sessions_df.head(5), len(valid_sessions_df)

(                                          prev_items   next_item locale
 0  ['B0B42XZNCW' 'B07ND7MKVZ' 'B0B42XZNCW' 'B07ND...  B07S43KJL7     UK
 1                        ['B09BW9Y1YR' 'B0B6Q3H47G']  B01EUNA3VS     UK
 2           ['B013KV0FS2' 'B06VW8Y81Y' 'B013KV0FS2']  B014WYVD1O     UK
 3           ['B09TB336FC' 'B083P3LGFY' 'B09TB3KFNV']  B09N3M5JNJ     UK
 4           ['B0B77H8NYS' 'B07HQ7P9BQ' 'B07MWYRVPG']  B009ARMB0Q     UK,
 94574)

In [53]:
valid_sessions_df.head(2)

,prev_items,next_item,locale
0,['B0B42XZNCW' 'B07ND7MKVZ' 'B0B42XZNCW' 'B07ND...,B07S43KJL7,UK
1,['B09BW9Y1YR' 'B0B6Q3H47G'],B01EUNA3VS,UK


In [57]:
valid_prediction_300_df = read_valid_300_prediction()
len(valid_prediction_300_df)

94574

In [58]:
filtered_valid_prediction_300_df = filter_hist(valid_sessions_df, valid_prediction_300_df, 300)

100%|██████████| 94574/94574 [01:00<00:00, 1565.90it/s]


In [10]:
if 'scores' in filtered_valid_prediction_300_df:
    filtered_valid_prediction_300_df.drop(columns=['scores'], inplace=True)

In [11]:
filtered_valid_prediction_300_df

,next_item_prediction
0,"[B0BGS6VHTF, B0BGS7PZCS, B08KQ96C3V, B0BGS8M5F..."
1,"[B014GLJOLO, B014GLJIAG, B014GLJI92, B014GLJI2..."
2,"[B07C5TQVG5, B001TV6DKK, B000AQY0KU, B01EPQODR..."
3,"[B09CTRHVQ2, B09CTQXZ4X, B0BFHQ8PLJ, B0BFF8PWL..."
4,"[B076JB96D8, B09HH6VN55, B09HH3F41F, B09HH1BSS..."
...,...
78324,"[B07HKS5SH7, B09H7DRWB3, B0BJNTCF3T, B000FQNK9..."
78325,"[B09VB3VVJM, B093H2DK1N, B0B9S6PSPH, B07QT24FX..."
78326,"[B01ABQLWR2, B08WQL1J4K, B08XBZD3DP, B08CH8YVJ..."
78327,"[B01M58CVJN, B0145YNPDA, B01M4FMS8I, B07X1JMFN..."


In [56]:
filtered_valid_prediction_300_df.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_xlm_roberta_results/valid_results/phase2_task1_valid_300_filtered.parquet', engine='pyarrow')

In [22]:
filtered_valid_prediction_150_df = copy.deepcopy(filtered_valid_prediction_300_df)
filtered_valid_prediction_150_df['next_item_prediction'] = filtered_valid_prediction_300_df['next_item_prediction'].apply(lambda x : x[:150])

In [23]:
filtered_valid_prediction_150_df.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_roberta_results_UK_new_data/valid_results/phase2_task1_valid_150_with_score_UK_filtered.parquet', engine='pyarrow')

# test prediction

In [61]:
product_data = read_product_data()

In [62]:
test_sessions_df = read_test_data()
test_sessions_df.head(2), len(test_sessions_df)

(                                 prev_items locale
 0  ['B087VLP2RT' 'B09BRQSHYH' 'B099KW4ZLV']     DE
 1  ['B08XW4W667' 'B096VMCJYF' 'B096VMCJYF']     DE,
 316972)

In [63]:
test_prediction_300_df = read_test_300_prediction()
len(test_prediction_300_df), len(test_prediction_300_df.iloc[0]['next_item_prediction'])

(316972, 350)

In [64]:
test_prediction_300_df.head(2)

,next_item_prediction,scores
0,"[B07GJXLBJ2, B078Y36QFJ, B099KW4ZLV, B08JW5DR7...","[289.486083984375, 289.4067687988281, 289.1253..."
1,"[B079NW6DQ1, B096VMCJYF, B0B2Z8DJYP, B079NVK7C...","[289.16607666015625, 288.5951843261719, 288.11..."


In [65]:
filtered_test_prediction_300_df = filter_hist(test_sessions_df, test_prediction_300_df, 300)

100%|██████████| 316972/316972 [08:23<00:00, 629.42it/s]


In [66]:
len(filtered_test_prediction_300_df.iloc[0]['next_item_prediction']), len(filtered_test_prediction_300_df.iloc[0]['scores'])

(300, 300)

In [ ]:
if 'scores' in filtered_test_prediction_300_df:
    filtered_test_prediction_300_df.drop(columns=['scores'], inplace=True)

In [46]:
filtered_test_prediction_300_df.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/phase2_task1_xlm_roberta_results/test_results/phase2_task1_test_300_filtered.parquet')

In [47]:
filtered_test_prediction_100_df = copy.deepcopy(filtered_test_prediction_300_df)
filtered_test_prediction_100_df['next_item_prediction'] = filtered_test_prediction_300_df['next_item_prediction'].apply(lambda x : x[:100])
len(filtered_test_prediction_100_df.iloc[0]['next_item_prediction'])

100

In [48]:
filtered_test_prediction_100_df['scores'] = filtered_test_prediction_300_df['scores'].apply(lambda x : x[:100])
len(filtered_test_prediction_100_df.iloc[0]['scores'])

100

In [49]:
filtered_test_prediction_100_df.to_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/candidates/roberta/phase2_roberta_task1_test_100_with_score.parquet')

# calculate metrics

In [59]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [25]:
UK_valid_sessions_df = valid_sessions_df[valid_sessions_df['locale'] == 'UK']

In [14]:
UK_valid_prediction_300_df = filtered_valid_prediction_300_df[valid_sessions_df['locale'] == 'UK']

In [15]:
DE_valid_sessions_df = valid_sessions_df[valid_sessions_df['locale'] == 'DE']
DE_valid_prediction_300_df = filtered_valid_prediction_300_df[valid_sessions_df['locale'] == 'DE']

In [26]:
JP_valid_sessions_df = valid_sessions_df[valid_sessions_df['locale'] == 'JP']
JP_valid_prediction_300_df = filtered_valid_prediction_300_df[valid_sessions_df['locale'] == 'JP']

In [27]:
cal_hit_and_mrr(DE_valid_sessions_df['next_item'], DE_valid_prediction_300_df['next_item_prediction'])

100%|██████████| 88913/88913 [00:04<00:00, 18732.61it/s]


(0.7123592725473216, 0.17543721424179237)

In [28]:
cal_hit_and_mrr(JP_valid_sessions_df['next_item'], JP_valid_prediction_300_df['next_item_prediction'])

100%|██████████| 78329/78329 [00:03<00:00, 20957.78it/s]


(0.7002515032746492, 0.2096176694089955)

In [29]:
cal_hit_and_mrr(UK_valid_sessions_df['next_item'], UK_valid_prediction_300_df['next_item_prediction'])

100%|██████████| 94574/94574 [00:05<00:00, 17954.39it/s]


(0.6977816313151606, 0.17564609772143364)

In [60]:
cal_hit_and_mrr(valid_sessions_df['next_item'], filtered_valid_prediction_300_df['next_item_prediction'])

100%|██████████| 94574/94574 [00:04<00:00, 19917.14it/s]


(0.7324105991075771, 0.2240871480011947)

In [ ]:
cal_hit_and_mrr(valid_sessions_df['next_item'], filtered_valid_prediction_150_df['next_item_prediction'])

In [24]:
product_data = read_product_data()
product_data = product_data.set_index('id')

In [78]:
def compare_title(index):
    print('********last item*********')
    prev_items = eval(valid_sessions_df.iloc[index]['prev_items'].replace(' ', ','))
    last_item = prev_items[-1]
    last_item_title = product_data.loc[last_item][['title', 'locale']]
    print(last_item_title)
    print('*******ground truth*******')
    next_item = valid_sessions_df.iloc[index]['next_item']
    next_item_title = product_data.loc[next_item][['title', 'locale']]
    print(next_item_title)
    print('*******prediction*********')
    for i in range(10):
        print(f'No. {i} : ')
        candidate = valid_prediction_df.iloc[index]['next_item_prediction'][i]
        predict_title = product_data.loc[candidate][['title', 'locale']]
        print(predict_title)

In [47]:
compare_title(200)

********last item*********
title     [5 Pcs] Stitch Makeup Brush Set, Lilo and Stit...
locale                                                   UK
Name: B09XHPY9PG, dtype: object
*******ground truth*******
title     Lilo and Stitch Temporary Tattoos 8 sheetsTemp...
locale                                                   UK
Name: B0B8MRC9HB, dtype: object
*******prediction*********
No. 0 : 
title     5Pcs Stitch Makeup Brush Set, Lilo and Stitch ...
locale                                                   UK
Name: B083NFMMF4, dtype: object
No. 1 : 
title     [5 Pcs] Stitch Makeup Brush Set Lilo and Stitc...
locale                                                   UK
Name: B0BCDYXLGD, dtype: object
No. 2 : 
title     [5 Pcs] Stitch Makeup Brush Set, Lilo and Stit...
locale                                                   UK
Name: B09XHPY9PG, dtype: object
No. 3 : 
title     Disney Stitch Hair Brush, Detangle Hair Brush ...
locale                                                   UK
Nam

In [48]:
compare_title(100)

********last item*********
title     Magnetic Eyelashes False Eyelashes Natural Loo...
locale                                                   UK
Name: B09T32YPZ8, dtype: object
*******ground truth*******
                                                        title locale
id                                                                  
B0B2VMQ213  False Eyelashes, Russian Strip Lashes D Curl V...     UK
B0B2VMQ213  Faux Cils 5 Styles Faux Cils Naturel Avec Coll...     FR
*******prediction*********
No. 0 : 
                                                        title locale
id                                                                  
B082FWPZ5Z  Künstliche Wimpern mit 3 Magneten, Dual Magnet...     DE
B082FWPZ5Z  Magnetic Eyelashes, Magnetic lashes, Magnetic ...     UK
No. 1 : 
title     Magnetic Eyelashes 5 Pares False Eyelashes wit...
locale                                                   UK
Name: B09Y5M5GLW, dtype: object
No. 2 : 
title     Grandeur Magnetic Eyelashe

In [49]:
pd.set_option('max_colwidth',1000)

In [50]:
valid_sessions_df.iloc[1000]

prev_items    ['B0BC8YRH6V' 'B0BC8YRH6V' 'B0BC1HCT56']
next_item                                   B09GB3BD7R
locale                                              JP
Name: 1000, dtype: object

In [51]:
compare_title(1000)

********last item*********
title     『2022冬新型』 セラミックヒーター 【知能恒温 メモリー機能】 セラミックファンヒーター 暖房器具 省エネ 電気ファンヒーター 首振り 3段階温度調整 800W/1200W 切り忘れ防止 二重過熱保護 転倒OFF タイマー機能 リモコン付き 大風量 速暖 電気ストーブ 電気ヒーター 足元 スリム タワー 脱衣所 寝室 トイレ 日本語取扱説明書 8畳 S720
locale                                                                                                                                                                                        JP
Name: B0BC1HCT56, dtype: object
*******ground truth*******
title     モダンデコ カーボンヒーター 電気 【保証1年間】 スリム 遠赤外線 足元 (ホワイト)
locale                                              JP
Name: B09GB3BD7R, dtype: object
*******prediction*********
No. 0 : 
title     『2022冬新型』 セラミックヒーター 【知能恒温 メモリー機能】 セラミックファンヒーター 暖房器具 省エネ 電気ファンヒーター 首振り 3段階温度調整 800W/1200W 切り忘れ防止 二重過熱保護 転倒OFF タイマー機能 リモコン付き 大風量 速暖 電気ストーブ 電気ヒーター 足元 スリム タワー 脱衣所 寝室 トイレ 日本語取扱説明書 8畳 S720
locale                                                                                                                                                       

In [52]:
compare_title(2000)

********last item*********
title     【EC限定】UHAグミサプリ鉄＆葉酸　60日分（10日分×6箱）
locale                                  JP
Name: B097X5WY6H, dtype: object
*******ground truth*******
title     【まとめ買い】グミサプリ　亜鉛＆マカ　30日分（60粒）　コーラ味　3個セット
locale                                         JP
Name: B09SKRW42Q, dtype: object
*******prediction*********
No. 0 : 
title     【EC限定】UHAグミサプリ鉄＆葉酸　60日分（10日分×6箱）
locale                                  JP
Name: B097X5WY6H, dtype: object
No. 1 : 
title     グミサプリ UHAグミサプリ マルチビタミン 30日分SP 白 60 粒
locale                                      JP
Name: B08Y655N98, dtype: object
No. 2 : 
title     ＵＨＡ味覚糖 ＵＨＡグミサプリ 鉄＆葉酸 ２０日分×３個
locale                              JP
Name: B01JKHKCOW, dtype: object
No. 3 : 
title     【まとめ買い】グミサプリ　鉄　20日分（40粒）　グレープ味　3個セット
locale                                      JP
Name: B09SKY29S2, dtype: object
No. 4 : 
title     UHAグミサプリ 鉄 20日分SP
locale                   JP
Name: B086X9CBRD, dtype: object


In [53]:
compare_title(5601)

********last item*********
                                                                                                                                                                           title  \
id                                                                                                                                                                                 
B08C4Q8QSK  Smart Steckdosenleiste für HomeKit WLAN Mehrfachsteckdose, Refoss mit 4 AC-Ausgänge und 4 USB-Anschlüsse, kompatibel für Apple Siri, Alexa, Google Assistant, 2,4Ghz   
B08C4Q8QSK                         Smart power strip WiFi multiple socket, Refoss with 4 AC outlets and 4 USB ports, compatible with Apple Siri, Alexa, Google Assistant, 2.4Ghz   

           locale  
id                 
B08C4Q8QSK     DE  
B08C4Q8QSK     UK  
*******ground truth*******
title     Smart Power Strip WiFi Plug - Refoss Smart Outlets Smart Extension Lead 1.8m with 4 AC Outlets, Compatible with Amazon Alexa, Google As

In [55]:
compare_title(45677)

********last item*********
title     21.5カタワイドエキショウディスプレイ KA220HQbmidx (ヒコウタク/1920x1080/ブラック/ミニD-Sub15ピン・DVI-D24ピン・HDMI/スピーカー/フリッカーフリー/BLフィルター)
locale                                                                                                            JP
Name: B01EL89H6E, dtype: object
*******ground truth*******
title     Acer ゲーミングモニター ディスプレイ SigmaLine 21.5インチ KG221QAbmix 1ms(GTG) 75Hz TN FPS向き フルHD FreeSync HDMI スピーカー内蔵 ブルーライト軽減
locale                                                                                                                JP
Name: B07ZJFLTBF, dtype: object
*******prediction*********
No. 0 : 
title     21.5カタワイドエキショウディスプレイ KA220HQbmidx (ヒコウタク/1920x1080/ブラック/ミニD-Sub15ピン・DVI-D24ピン・HDMI/スピーカー/フリッカーフリー/BLフィルター)
locale                                                                                                            JP
Name: B01EL89H6E, dtype: object
No. 1 : 
title     21.5カタワイドエキショウディスプレイ KA220HQbid (TN/ヒコウタク/1920x1080/200cd/100000000:1/5ms/ブラック/ミニD-Su

In [63]:
valid_prediction_df.iloc[45677]

next_item_prediction    [B00ZI9232Q, B07WNTJMW5, B09CYKQ724, B0B2ZCSDN3, B088HD6R5P, B086LMJKVY, B07V5BHZLW, B07XVR3XN2, B09NVXJY9M, B07V47WQD8, B0B2QPW4NM, B0989RJBY8, B094CCD62V, B088HBN5SJ, B09F3HR4BT, B088HF6Q3M, B0969CY544, B094CFGW3R, B095HH3G7N, B07NH8S9JL, B07XVQGH7X, B0973GF3F5, B09RPFXGJF, B0B14N36PS, B09F3X3P7P, B08KJ97ZJY, B07JPXTL28, B08GFWHRJC, B0B4QQT3YV, B08GBVWCR3, B0BDY3NQS7, B08HGNYMWZ, B094D5J7T8, B089QF5VLX, B0B3WTZM5S, B0973TSW7W, B00DCGO2PS, B07KM8RD34, B09T5H8TW1, B09QPHY4DK, B0B6NBJ68L, B085TWLMF9, B08HGHJ98B, B09CYBJ31X, B08HH3BQMF, B07NWT82B5, B085Y5BB1L, B088HBM7HQ, B08BRX2RKY, B01L8H204W, B0B13MP97X, B0BL3HPFWD, B09LH6Y1TQ, B07PQF1MQC, B09F4FJBF5, B091T1TY9L, B09TQCWMQB, B07DQPQKC8, B07J9WRFTR, B09WYWLD8L, B01FQ6JMDS, B07WPY8P3L, B0989THPPX, B08BRX9K95, B09WND7Y7J, B0B7QYHRP5, B01LMGK61S, B0B8Y5SLBF, B08ZSLDD77, B091T7ZSWH, B00FEZD86O, B0BCGQBZQZ, B0B2QNGHSK, B08ZYKGLZX, B08HN39MML, B09SY6FL4W, B09VGGKZDZ, B08HCSQCH3, B09BHG1RT7, B09PB3YC8N, B091SMKJC3, B09

In [64]:
compare_title(45647)

********last item*********
title     KIMO インパクトドライバー セット 20V 200N·m 高回転 電動インパクトドライバー ブラシレス 4モード 無段変速・正逆転両用 充電式インパクトドライバー LEDライト付き 過負荷保護 過熱保護 DIY 日曜大工 家具の組立て インパクトドライバー ビット6本 ソケット4本 2.0Ahバッテリー バッグ 日本語説明書付き PSE認証済み QM-3602B
locale                                                                                                                                                                                          JP
Name: B08L4TX8Q4, dtype: object
*******ground truth*******
title     靴下 メンズ ビジネスソックス 8足セット 綿 抗菌防臭 高通気性 24-28cm 銀イオン加工 吸汗 耐摩耗性 黒 四季適用
locale                                                                 JP
Name: B0866FBJTR, dtype: object
*******prediction*********
No. 0 : 
title     インパクトレンチ 電動インパクトレンチ コードレスインパクトレンチ 一台３役 ドリルドライバー インパクトドライバー 最大トルク600N·m マキタ18Vバッテリー対応可能 無段変速 ブラシレスモーター 正逆転両用 回転最大数:3300r/min 自動停止モード タイヤ交換 ホームメンテナンス ボルトの緩み ホイール交換 バッテリー・ケース付き
locale                                                                                                                      

In [65]:
compare_title(65647)

********last item*********
title     デルフォニックス ロルバーン ポケット付メモ [リング式/切離しミシン目付き] A5 イエロー NRP08-YE
locale                                                          JP
Name: B007B8N2RC, dtype: object
*******ground truth*******
title     デルフォニックス ロルバーン ポケット付メモ [リング式/切離しミシン目付き] A5 ダークブルー NRP08-DB
locale                                                            JP
Name: B007B8N2YA, dtype: object
*******prediction*********
No. 0 : 
title     デルフォニックス ロルバーン ポケット付メモ [リング式/切離しミシン目付き] A5 ダークブルー NRP08-DB
locale                                                            JP
Name: B007B8N2YA, dtype: object
No. 1 : 
title     デルフォニックス ロルバーン ポケット付メモ [リング式/切離しミシン目付き] A5 ライトブルー(新タイプ) NRP08-SB
locale                                                                  JP
Name: B01DPSRA88, dtype: object
No. 2 : 
title     デルフォニックス ロルバーン ポケット付メモ [リング式/切離しミシン目付き] A5 ローズ NRP08-RO
locale                                                         JP
Name: B00CHI7Y52, dtype: object
No. 3 : 
title     デルフォニックス ロルバーン ポケット付メ

In [66]:
compare_title(65747)

********last item*********
title     USB Plug Charger,Rekavin Dual Port USB Wall Plug Adapter UK Compact Mains Charge 2.1A with Smart IC Charging Technology for iPhone 11 Xs/XS Max/XR/X/8/7/6/Plus,iPad Pro/Air 2/Mini 4 etc
locale                                                                                                                                                                                           UK
Name: B08L6JJ9M2, dtype: object
*******ground truth*******
title     USB Plug UK Mains Charger,Rekavin Dual USB Plug Adapter UK Compact Wall Charge Head 2.1A with Smart IC Fast Charging Technology for iPhone 13 12 11 Xs/XS Max/XR/X/8/7/6/Plus,iPad Pro/Air 2/Mini 4 etc
locale                                                                                                                                                                                                         UK
Name: B09K6C8ZWP, dtype: object
*******prediction*********
No. 0 : 
title     USB Plug Charger,Rekavin

In [67]:
compare_title(65847)

********last item*********
title     Merriway BH01938 100mm (4 inch) Sink Plunger with 225mm (9 inch) Plastic Handle Diameter, White/Black
locale                                                                                                       UK
Name: B00TQWT2WK, dtype: object
*******ground truth*******
title     O.R.S Hydration Tablets with Electrolytes, Vegan, Gluten and Lactose Free Formula – Soluble Sports Hydration Tablets with Natural Blackcurrant Flavour, 24 Tablets
locale                                                                                                                                                                    UK
Name: B00NGTCKWS, dtype: object
*******prediction*********
No. 0 : 
title     Merriway BH03116 Sink Plunger 150mm (6 inch) Dia. with 225mm (9 inch) Plastic Handle
locale                                                                                      UK
Name: B00OBFZF5A, dtype: object
No. 1 : 
title     PLUMB-PAK T627AMZ Mini Sink Plunger

In [69]:
compare_title(65947)

********last item*********
title     【BLKP】 パール金属 保温 弁当箱 ランチ ジャー 限定 ブラック 1060 ご飯茶碗 約2.7杯分 どんぶり BLKP 黒 AZ-5025
locale                                                                          JP
Name: B07S4W67QT, dtype: object
*******ground truth*******
title     【BLKP】 パール金属 スープジャー 270ml 限定 ブラック フードジャー 保温 弁当箱 BLKP 黒 AZ-5026
locale                                                                JP
Name: B07QMRS6KL, dtype: object
*******prediction*********
No. 0 : 
title     タイガー 魔法瓶 保温 弁当箱 ステンレス ランチ ジャー 茶碗 約 3 杯分 ブラック LWU-A172-KM Tiger
locale                                                                JP
Name: B01K1QPFB6, dtype: object
No. 1 : 
title     パール金属 ランチジャー 1500 弁当箱 どんぶり 全真空 ステンレス 保温 保冷 ブラック ホームレーベル HB-5668
locale                                                                 JP
Name: B09QFRNP86, dtype: object
No. 2 : 
title     タイガー 魔法瓶 保温 弁当箱 ステンレス ランチ ジャー 茶碗 約 4 杯分 ブラック LWU-A202-KM Tiger
locale                                                                JP
Name: B01K1Q

In [70]:
compare_title(65748)

********last item*********
title     おつまみセット 詰め合わせ 12種12品 Aセット (おつまみセット)
locale                                     JP
Name: B0BCX4ZG9C, dtype: object
*******ground truth*******
title     駄菓子 お菓子 詰め合わせ プレゼント 食べ比べ アソート 50種約55個入 (駄菓子詰め合わせ)
locale                                                   JP
Name: B0B566J6X4, dtype: object
*******prediction*********
No. 0 : 
title     おつまみセット 詰め合わせ お楽しみランダムセット 10食入り
locale                                 JP
Name: B08CZNFQVT, dtype: object
No. 1 : 
title     おつまみセット 詰め合わせ お楽しみバラエティーセット 12食入り
locale                                   JP
Name: B08LN3MLW3, dtype: object
No. 2 : 
title     大人の厳選 おつまみ 30点セット 珍味 駄菓子 詰め合わせ 人気 お楽しセット まとめ買い・大人買い
locale                                                     JP
Name: B09BBH7KFL, dtype: object
No. 3 : 
title     グリコ おつまみセット15品　ジョッキ型ＢＯＸ入り
locale                           JP
Name: B071ZBC8XR, dtype: object
No. 4 : 
title     おつまみセット 詰め合わせ 人気 ギフト 世界の珍味シリーズ 8種セット おしゃれ 高級 珍味 お取り寄せ
locale                                

In [71]:
compare_title(65758)

********last item*********
                                                                                                                                                                                title  \
id                                                                                                                                                                                      
B07QKXWP26           Odoland Campingkocher 3500W Gaskocher mit Piezozündung, Winddicht Gas Campingkocher für alle Schraubkartusche Gasbrenner, mit Adapter Faltbar Camping Kocher Rot   
B07QKXWP26  Odoland 3500W Windproof Camping Gas Stove Portable Collapsible Outdoor Camping Stove with Piezo Ignition Backpacking Stove for Outdoor Cooking Hiking Picnic and Trekking   

           locale  
id                 
B07QKXWP26     DE  
B07QKXWP26     UK  
*******ground truth*******
title     ROTHENBERGER Industrial Hockerkocher inkl. 1,5 m Propan-Gas-Schlauch & Propan-Konstantregler für Camping, Outdoor, 

In [72]:
compare_title(65768)

********last item*********
title     SINOPHANT High Waisted Leggings for Women, Buttery Soft Elastic Opaque Tummy Control Leggings, Plus Size Workout Gym Yoga Stretchy Pants (Navy1,L-XL)
locale                                                                                                                                                       UK
Name: B07Q34R58W, dtype: object
*******ground truth*******
title     SINOPHANT High Waisted Leggings for Women, Buttery Soft Elastic Opaque Tummy Control Leggings, Plus Size Workout Gym Yoga Stretchy Pants (Plum purple1,L-XL)
locale                                                                                                                                                              UK
Name: B07SL9XKN1, dtype: object
*******prediction*********
No. 0 : 
                                                                                                                                                                                                

In [73]:
compare_title(65778)

********last item*********
title     Keine Werbung Aufkleber - Schild – Folie - Sticker (Stop Bitte Keine Kostenlose Zeitung, Reklame, Flyer, Handzettel, Wurfsendungen, Wochenblätter, Werbung) für den Briefkasten - 1 Stück
locale                                                                                                                                                                                           DE
Name: B091GKW3K2, dtype: object
*******ground truth*******
title     5 x Keine Werbung Aufkleber - Schild extra starke Folie - Sticker (Stop Bitte Keine Kostenlose Zeitung, Reklame, Flyer, Handzettel, Wurfsendungen, Wochenblätter, Werbung) für den Briefkasten - 5 Stück
locale                                                                                                                                                                                                          DE
Name: B091FSFR2D, dtype: object
*******prediction*********
No. 0 : 
title     5 x Keine Werbung Aufk

In [74]:
compare_title(65779)

********last item*********
title     Hakenleiste TALIDA Edelstahl selbstklebend mit 3M Klebepad (Länge: 735 mm / Stangen-Ø 12 mm) inkl. 3 S-Haken für Küche und Bad Küchenhaken von SO-TECH®
locale                                                                                                                                                         DE
Name: B07Y1RKVP4, dtype: object
*******ground truth*******
                                                                                                                                                                       title  \
id                                                                                                                                                                             
B09DL9MFNH           Joejis Hakenleiste Küche - Küchenstange - Starker Selbstklebend 3M Ohne Bohren - Küchenleiste Stange für Küchenutensilien -Küchenhelfer Leiste - Silber   
B09DL9MFNH  Joejis Kitchen Utensil Rack 40cm - Aluminium Kitch

In [75]:
compare_title(65789)

********last item*********
title     3M Extension Lead with USB Slots, 4 Way 2 USB Extension Socket with Individual Switches Mountable Power Strips with 3M Extension cable 3250W
locale                                                                                                                                              UK
Name: B09CTTXVN5, dtype: object
*******ground truth*******
title     Masterplug BSG3-MP Four Socket Long Extension Lead, 3 Metres, 13 Amp, White
locale                                                                             UK
Name: B0030WRXDM, dtype: object
*******prediction*********
No. 0 : 
title     Pro Elec pl15152 2 Gang 3 m Extension Lead - White
locale                                                    UK
Name: B078Y4JNXP, dtype: object
No. 1 : 
title     Masterplug BTG5-MP Two Socket Extension Lead, 5 Metres, White
locale                                                               UK
Name: B0036162JI, dtype: object
No. 2 : 
title     Mscien Extensi

In [76]:
compare_title(65790)

********last item*********
title     システムK カーペット ラグ ラグマット 防ダニ 抗菌 防臭 洗える ふわふわ肌触り 滑り止めホットカーペット対応 約3畳 トライアングルイエロー 200×250cm
locale                                                                                     JP
Name: B09S5ZXZCX, dtype: object
*******ground truth*******
title     システムK カーペット ラグ ラグマット 防ダニ 抗菌 防臭 洗える ふわふわ肌触り 滑り止めホットカーペット対応 約2畳 トライアングルイエロー 185×185cm
locale                                                                                     JP
Name: B09S5MNZFK, dtype: object
*******prediction*********
No. 0 : 
title     システムK カーペット ラグ ラグマット 防ダニ 抗菌 防臭 洗える ふわふわ肌触り 滑り止めホットカーペット対応 約2畳 ダークブラウン 185×185cm
locale                                                                                 JP
Name: B00QSVJVPA, dtype: object
No. 1 : 
title     システムK カーペット ラグ ラグマット 防ダニ 抗菌 防臭 洗える ふわふわ肌触り 滑り止めホットカーペット対応 約2畳 ダークグレー 185×185cm
locale                                                                                JP
Name: B081DSRWFR, dtype: object
No. 2 : 
title     システムK カーペット ラグ ラグマット

In [79]:
compare_title(65791)

********last item*********
title     NUK Babyphone mit Kamera Eco Control+ Video mit Gegensprechfunktion und Temperatursensor, frei von hochfrequenter Strahlung im Eco-Mode
locale                                                                                                                                         DE
Name: B007CK1ESS, dtype: object
*******ground truth*******
title     Ersatzkopf für Elektrorasierer,Scherkopf Ersatzteil Passend für Braun 7 Series,Ersatzklingen Folienkopf Zubehör für Rasierer（70S）
locale                                                                                                                                   DE
Name: B0B373TQW1, dtype: object
*******prediction*********
No. 0 : 
                                                                                                                                                                         title  \
id                                                                                              

In [80]:
compare_title(65792)

********last item*********
title     Milka Weihnachts-Kugeln Alpenmilch 1 x 100g, Zartschmelzende Alpenmilch Schokolade
locale                                                                                    DE
Name: B0161AA3AY, dtype: object
*******ground truth*******
title     Milka Weihnachts-Kugeln Weiß 1 x 100g, Weiße zartschmelzende Schokolade
locale                                                                         DE
Name: B0161A9YMW, dtype: object
*******prediction*********
No. 0 : 
title     Bahlsen Winter-Mix – 1er Pack Thekendispenser – Mit Klassikern wie Lebkuchen und Zimtsternen – Schokolade - einzeln verpackt (1 x 1,03 kg)
locale                                                                                                                                            DE
Name: B07R35GKCK, dtype: object
No. 1 : 
                                                                                                                     title  \
id                             

In [81]:
compare_title(65793)

********last item*********
title     DHC ビタミンBミックス トクヨウ90ニチブン
locale                          JP
Name: B01N47PAGY, dtype: object
*******ground truth*******
title     サプリクラフト ナイアシンアミド （ビタミンB3） 250mg 120カプセル
locale                                         JP
Name: B0893SSNRN, dtype: object
*******prediction*********
No. 0 : 
title     ネイチャーメイド ビタミンB6 20mg 80つぶ
locale                           JP
Name: B000FQTQO0, dtype: object
No. 1 : 
title     DHC ビタミンD 60ニチブン
locale                  JP
Name: B08L7TB1LQ, dtype: object
No. 2 : 
title     ビタミンD 30日分
locale            JP
Name: B07S376VHR, dtype: object
No. 3 : 
title     ネイチャーメイド スーパービタミンD(1000I.U.) 90ツブ
locale                                   JP
Name: B009OB2FVS, dtype: object
No. 4 : 
title     ネイチャーメイド ビタミンD(400I.U.) 60ツブ
locale                              JP
Name: B009PKQ5W8, dtype: object
No. 5 : 
title     ネイチャーメイド ビタミンC　500mg　　　　　　　
locale                             JP
Name: B000FQTQW2, dtype: object
No. 6 : 
title     ネイチャーメイドスー

In [82]:
compare_title(65794)

********last item*********
title     Official SMOK RPM2 2000mAh 80W E Cigarette Vape Starter Kit, USB Type-C Fast Charging ecig, Large 1.14" TFT Screen, 2X Pods 2ml, Mesh 0.16ohm & 0.4ohm, Real Pod Mod, No Nicotine - Prism Blue
locale                                                                                                                                                                                                UK
Name: B08FM96DQS, dtype: object
*******ground truth*******
title     Official SMOK RPM2 2000mAh 80W E Cigarette Vape Starter Kit, USB Type-C Fast Charging ecig, Large 1.14" TFT Screen, 2X Pods 2ml, Mesh 0.16ohm & 0.4ohm, Real Pod Mod, No Nicotine - Prism Rainbow
locale                                                                                                                                                                                                   UK
Name: B08FM6CZYY, dtype: object
*******prediction*********
No. 0 : 
title     Official SMOK RPM2 2000mAh

In [83]:
compare_title(65800)

********last item*********
title     Nitrilhandschuhe 100 Stück Box (L, Schwarz) Einweghandschuhe, Einmalhandschuhe, Untersuchungshandschuhe, Nitril Handschuhe, puderfrei, ohne Latex, unsteril, latexfrei, disposible gloves, black
locale                                                                                                                                                                                                  DE
Name: B00X81M4FW, dtype: object
*******ground truth*******
title     Nitrilhandschuhe 100 Stück Box (S, Schwarz) Einweghandschuhe, Einmalhandschuhe, Untersuchungshandschuhe, Nitril Handschuhe, puderfrei, ohne Latex, unsteril, latexfrei, disposible gloves, black
locale                                                                                                                                                                                                  DE
Name: B00X81GP9I, dtype: object
*******prediction*********
No. 0 : 
title     Nitrilhandschuhe 100 Stü

In [84]:
compare_title(65801)

********last item*********
title     【ビール 糖質ゼロ】キリン一番搾り 糖質ゼロ 350ml×24本
locale                                  JP
Name: B08F81KS91, dtype: object
*******ground truth*******
title     【ビール 糖質ゼロ】キリン一番搾り 糖質ゼロ 350ml×24本
locale                                  JP
Name: B08F81KS91, dtype: object
*******prediction*********
No. 0 : 
title     【ビール 糖質ゼロ】キリン一番搾り 糖質ゼロ 500ml×24本
locale                                  JP
Name: B08F5KS1WF, dtype: object
No. 1 : 
title     【ビール 糖質ゼロ】キリン一番搾り 糖質ゼロ 350ml×6本
locale                                 JP
Name: B08GM8Z41N, dtype: object
No. 2 : 
title     【ノンアルコールビール 脂肪の吸収を抑え、糖の吸収をおだやかに】キリン パーフェクトフリー 350ml×24本
locale                                                         JP
Name: B00XVEBHRW, dtype: object
No. 3 : 
title     【第3の ビール 新ジャンル】キリン 濃い味〈糖質ゼロ〉 500ml×24本
locale                                        JP
Name: B0052QSVKS, dtype: object
No. 4 : 
title     【第3の ビール 新ジャンル】キリン 濃い味〈糖質ゼロ〉 350ml×24本
locale                                        JP
Name: B004P9

In [85]:
compare_title(65802)

********last item*********
title     【精米】[Amazon限定ブランド] 580.com 新潟県産 無洗米 コシヒカリ 5kg 令和4年産
locale                                                     JP
Name: B08MWS3M85, dtype: object
*******ground truth*******
title     [Amazon限定ブランド] Oseamaid バスタオル 大判 5色5 枚セット マイクロファイバー タオル 吸水速乾抗菌防臭(白 ベージュ グレー カーキ色 コーヒー色)
locale                                                                                         JP
Name: B091P3LWM5, dtype: object
*******prediction*********
No. 0 : 
title     【セイマイ】[アマゾンゲンテイブランド] 580.com アイヅサン ムセンマイ コシヒカリ 2kg レイワ3ネンサン
locale                                                             JP
Name: B0777HL1GV, dtype: object
No. 1 : 
title     【セイマイ】[アマゾンゲンテイブランド] 580.com ミヤギケンサン ムセンマイ ツヤヒメ 5kg レイワ3ネンサン
locale                                                              JP
Name: B07NGBBLTH, dtype: object
No. 2 : 
title     【セイマイ】[アマゾンゲンテイブランド] 580.com ミヤギケンサン ハクマイ ツヤヒメ 5kg レイワ3ネンサン
locale                                                             JP
Name: B00WJNO21E, dtyp

In [86]:
compare_title(65803)

********last item*********
title     洗面台 排水口 ごみ受け パンチング 排水口サイズ：約35-45mm 流し用 排水口カバー 抗菌 ゴミカゴ 排水フィルター 18-8ステンレス 浅型 ゴミカゴ 洗面台 洗面所 (XS(排水口サイズ：約35-45mm))
locale                                                                                                               JP
Name: B0B9KXSLQH, dtype: object
*******ground truth*******
title     ユニットバス ステンレス ごみ受け 排水口サイズ：約58-80mm 排水溝 ごみ受け 流し用 排水口カバー 抗菌 キッチン ヘアキャッチャー シンク ネット 髪の毛 取り パンチング 排水口 ゴミ受け 8cm (M(排水口サイズ：約58-80mm))
locale                                                                                                                               JP
Name: B0B9KVDRQ4, dtype: object
*******prediction*********
No. 0 : 
title     洗面台排水口用 ゴミ受け パンチング ステンレス 排水口サイズ：35-45mm 2個セット
locale                                               JP
Name: B07MK8MGPM, dtype: object
No. 1 : 
title     洗面台排水口用 ゴミ受け パンチング ステンレス 排水口サイズ：35-45mm
locale                                         JP
Name: B07SYJ91K5, dtype: object
No. 2 : 
title     洗面器排水口用 洗面台 パンチング ゴミ受け 18 – 8ス

In [87]:
compare_title(65807)

********last item*********
                                                                                                                                                                                           title  \
id                                                                                                                                                                                                 
B008FN6YOU  Optimum Nutrition ON Optimen Multivitamin Kapseln, Vitamine und Mineralstoffe Tabletten für Männer mit Zink, Magnesium und Aminosäuren Komplex Hochdosiert, 60 Portionen 180 Kapseln   
B008FN6YOU                                           Optimum Nutrition Opti-Men Multivitamin Supplements for Men with Vitamin D, Vitamin C, Vitamin A and Amino Acids, 60 Servings, 180 Capsules   

           locale  
id                 
B008FN6YOU     DE  
B008FN6YOU     UK  
*******ground truth*******
                                                                                 

In [88]:
compare_title(65810)

********last item*********
title     1ショクブンノヤサイガトレルグーグーキッチン【9カゲツゴロカラ】オススメセット
locale                                         JP
Name: B093BSRPJY, dtype: object
*******ground truth*******
title     【パンツ Mサイズ】メリーズ オムツ さらさらエアスルー (6~11kg) 74枚 【Amazon.co.jp限定】
locale                                                            JP
Name: B07H7D3CG7, dtype: object
*******prediction*********
No. 0 : 
title     ワコウドウ グーグーキッチン トリゾウスイ
locale                       JP
Name: B01KLW8IO6, dtype: object
No. 1 : 
title     ワコウドウ グーグーキッチン トリトヤサイノチュウカアンカケ
locale                                JP
Name: B01KLW9C46, dtype: object
No. 2 : 
title     和光堂 具たっぷりグーグーキッチン [12か月頃から] おすすめセット 6種×2袋(12袋) ベビーフード
locale                                                       JP
Name: B0BD7X1JRJ, dtype: object
No. 3 : 
title     ワコウドウ グーグーキッチン ダイコントブタノカクニ
locale                            JP
Name: B01KLW8P1W, dtype: object
No. 4 : 
title     ワコウドウ グーグーキッチン ツナトキノコノリゾット
locale                            JP
Name: B01KLW95UW, dtyp

In [89]:
compare_title(65811)

********last item*********
title     Replacement Toothbrush Heads for Oral-B,Replacement Heads Compatible with Oral B Braun Electric Toothbrush (Pack of 12)
locale                                                                                                                         UK
Name: B08YYBQYCQ, dtype: object
*******ground truth*******
title     SINOPHANT High Waist Knickers for Women, Ladies Cotton Knickers Full Briefs Multipack
locale                                                                                       UK
Name: B091GGW93Q, dtype: object
*******prediction*********
No. 0 : 
title     Electric Toothbrush Head for Oral-B, 12 Pack Replacement Toothbrush Heads Compatible with Most Braun Oral B Electric Toothbrushes, Classic Precision Clean Replacement Brush Heads
locale                                                                                                                                                                                    UK
Name: B0BH3RWZ9N

In [90]:
compare_title(65812)

********last item*********
title     HooMoo T16 LED バックランプ 爆光 1200LM高輝度 無極性 キャンセラー内蔵 CANBUS 24連3030 SMD LED バックアップ 後退灯 7000K ホワイト 2個セット
locale                                                                                                    JP
Name: B07GLNVGJ5, dtype: object
*******ground truth*******
title     HooMoo T16 LED バックランプ 爆光 1200LM高輝度 無極性 キャンセラー内蔵 CANBUS 24連3030 SMD LED バックアップ 後退灯 7000K ホワイト 4個セット
locale                                                                                                    JP
Name: B07QGR8725, dtype: object
*******prediction*********
No. 0 : 
title     OXILAM T16 LED, バックランプ 爆光 後退灯 キャンセラー内蔵 Canbus T15 W16W バックランプ 無極性 ランプセット ホワイト (2個入)
locale                                                                                     JP
Name: B07KPXR9P5, dtype: object
No. 1 : 
title     T16 LED バックランプ 爆光1200ルーメン キャンセラー内蔵 CANBUS バックランプ T16 W16W T10 T15 バックランプ 高輝度 ホワイト4014LED 45連 12V 無極性 ホワイト 後退灯 バックライト 6000K 50000時間以上寿命 (4個セット)
locale                      

In [91]:
compare_title(65813)

********last item*********
title     Yale YKB/550/DB1 Large Electronic Key Cabinet, for Up to 100X Keys, Includes Numbered Racks, Supplied with 2 Override Keys
locale                                                                                                                            UK
Name: B00VYSHUYG, dtype: object
*******ground truth*******
                                                                                                                                                                                                        title  \
id                                                                                                                                                                                                              
B006XBSPF2               Yale YSV/200/DB1 Kleiner Werttresor, digitale Tastatur, LED-Leuchtanzeigen, 15-mm-Stahlverriegelungsbolzen, Notbetätigungsschlüssel, Wand- und Bodenbefestigung, schwarze Ausführung   
B006XBSPF2  Yale YSV/

In [93]:
compare_title(65814)

********last item*********
                                                                                                                                                                   title  \
id                                                                                                                                                                         
B093SHYNVR     SUTAN Elektrische Heizdecke, 180*130cm, 6 Heizstufen, Heizdecke mit abschaltautomatik, Elektrische Wärmedecke Überhitzungsschutz, Maschinenwaschbar, Grau   
B093SHYNVR                                           SUTAN Manta eléctrica , 180x130cm, 6 Niveles de Temperatura, Apagado automático en 3 Horas, Lavable a máquina, Gris   
B093SHYNVR  SuTan Coperta Elettrica Termica, 180X130cm, 6 Livelli di Temperatura, Spegnimento Automatico, Protezione dal Surriscaldamento, Lavabile in Lavatrice, Grigio   

           locale  
id                 
B093SHYNVR     DE  
B093SHYNVR     ES  
B093SHYNVR     IT  
*******groun

In [94]:
compare_title(65815)

********last item*********
title     サーモス 交換用部品 ケータイマグ JNL パッキンセット (フタパッキン・せんパッキン)
locale                                               JP
Name: B00AEIIBVI, dtype: object
*******ground truth*******
title     サーモス 交換用部品 ケータイマグ JNO-250/350 パッキンセット (フタパッキン・せんパッキン)
locale                                                       JP
Name: B00GN711NE, dtype: object
*******prediction*********
No. 0 : 
title     サーモス 交換用部品 ケータイマグ JOK パッキンセット (フタパッキン・せんパッキン)
locale                                               JP
Name: B08NSNZKLQ, dtype: object
No. 1 : 
title     サーモス 交換用部品 ケータイマグ JNR パッキンセット (フタパッキン・せんパッキン)
locale                                               JP
Name: B07MQC71M4, dtype: object
No. 2 : 
title     サーモス 交換用部品 ケータイマグ JNI パッキンセット (フタパッキン・せんパッキン)
locale                                               JP
Name: B00A9OJOTA, dtype: object
No. 3 : 
title     サーモス 交換用部品 ケータイマグ JNO-500 パッキンセット (フタパッキン・せんパッキン)
locale                                                   JP
Name: B00GN6ZB7C, dtype: ob

In [95]:
compare_title(65819)

********last item*********
title     iwaki レンジのポット・チャキ モモイロ 400ml
locale                              JP
Name: B0078XFM1E, dtype: object
*******ground truth*******
title     iwaki パイレックス レンジのポット・チャキ グリーン 400ml 863-G
locale                                           JP
Name: B003AQBDWU, dtype: object
*******prediction*********
No. 0 : 
title     iwaki パイレックス レンジのポット・チャキ グリーン 400ml 863-G
locale                                           JP
Name: B003AQBDWU, dtype: object
No. 1 : 
title     iwaki Pyrex レンジノポット プチティー ブルー 400ml
locale                                     JP
Name: B0009MZ5AE, dtype: object
No. 2 : 
title     iwaki パイレックス 【レンジノポット】 ポット&ドリッパー ブラック 1L 7966C-BK2
locale                                                    JP
Name: B002UKPA7Q, dtype: object
No. 3 : 
title     iwaki パイレックス ジャンピングティーポット ブラック 640ml 894T-BK
locale                                              JP
Name: B002UKPAAI, dtype: object
No. 4 : 
title     iwaki パイレックス 【レンジノポット】 コーヒー1000 ブラック 1L 7966-BK2
locale       

In [96]:
compare_title(65820)

********last item*********
title     Reayos Halloween String Lights, 2 Pack Total 20 Ft 40 LED Fall Garland with Decorations Lights, Pumpkin Maple Leaf String Lights for Halloween Thanksgiving Christmas Party Indoor Outdoor Decor
locale                                                                                                                                                                                                  UK
Name: B09DSF7MXR, dtype: object
*******ground truth*******
title     kuou 5 PCS Halloween Decoration Banners, Halloween Paper Chain Garland Banner Set Pumpkin Ghost Bat Spooky Spiders Skeleton Banners
locale                                                                                                                                     UK
Name: B08J2MKCG5, dtype: object
*******prediction*********
No. 0 : 
                                                                                                                                                            

In [97]:
compare_title(65821)

********last item*********
                                                                        title  \
id                                                                              
B07KPTB56T  Amazon Basics - Elektrischer Wasserkocher aus Glas, 1,7 l, 2200 W   
B07KPTB56T                   Amazon Basics 1.7 L 2200 W Electric Glass Kettle   

           locale  
id                 
B07KPTB56T     DE  
B07KPTB56T     UK  
*******ground truth*******
                                                                        title  \
id                                                                              
B07KPTB56T  Amazon Basics - Elektrischer Wasserkocher aus Glas, 1,7 l, 2200 W   
B07KPTB56T                   Amazon Basics 1.7 L 2200 W Electric Glass Kettle   

           locale  
id                 
B07KPTB56T     DE  
B07KPTB56T     UK  
*******prediction*********
No. 0 : 
title     SEVERIN WK 3417 Wasserkocher (ca. 2.200 W, 1 L) fire red metallic /schwarz
locale         